# Convolutional Neural Networks For Image Recognition

## Setup and Imports

In [1]:
# Setup
from __future__ import division, print_function, unicode_literals

# Imports
import numpy as np
import numpy.random as rnd
import os

# To make make consistent across code blocks
rnd.seed(42)

# Ploting
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Saving Parameters
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "cnn"

def save_fig(fig_id, tight_layout=True):
    path = os.path.join(PROJECT_ROOT_DIR, "data", CHAPTER_ID, fig_id + ".png")
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format='png', dpi=300)

**Import Keras**

In [2]:
import keras as kr

Using TensorFlow backend.


Couldn't import dot_parser, loading of dot files will not be possible.


# Convolutional Neural Network

## Import Dataset

In [3]:
from keras.datasets import cifar100
(x_train, y_train), (x_test, y_test) = cifar100.load_data(label_mode='fine')

In [4]:
def to_normalized(X):
    X = (X-128)/128
    

def to_one_hot(X):
    n_values = np.max(X) + 1
    y = np.eye(n_values)[X[:,0]]
    return y

In [5]:
y_train_adj = to_one_hot(y_train)
y_test_adj  = to_one_hot(y_test)

In [6]:
print(np.shape(x_train))

(50000, 32, 32, 3)


## Coarse Training

** Defining Structure**

In [7]:
def indep_layers(X, set_index):
    s = '__indep_layer_'
    net = Conv2D(48, (4, 4), strides=(1, 1), padding='same', activation='elu', name=set_index+s+str(1))(X)
    net = MaxPooling2D((2, 2), padding='valid')(net)
    net = Conv2D(64, (2, 2), strides=(1, 1), padding='same', activation='elu', name=set_index+s+str(2))(net)
    net = Dropout(0.50)(net)
    net = Conv2D(64, (2, 2), strides=(1, 1), padding='same', activation='elu', name=set_index+s+str(3))(net)
    net = Flatten()(net)

    net = Dense(128, activation='elu', name=set_index+s+str(4))(net)
    net = Dropout(0.50)(net)
    net = Dense(128, activation='elu', name=set_index+s+str(5))(net)
    net = Dropout(0.50)(net)
    net = Dense(100, activation='softmax')(net)
    
    return net

In [8]:
from keras.layers import Conv2D, MaxPooling2D, Input, Flatten, Dense, Dropout, Multiply, Add, Dot, RepeatVector, Reshape
from keras.optimizers import SGD
from keras.models import Model

coarse_categories = 20
fine_categories = 100
sigma = .01

input_img = Input(shape=(32, 32, 3), dtype='float32', name='main_input')

## SHARED LAYERS
net_shared = Conv2D(8, (4, 4), strides=(1, 1), padding='same', activation='elu')(input_img)
net_shared = MaxPooling2D((2, 2), padding='valid')(net_shared)

net_shared = Conv2D(16, (4, 4), strides=(1, 1), padding='same', activation='elu')(net_shared)
net_shared = MaxPooling2D((2, 2), padding='valid')(net_shared)

net_shared = Conv2D(32, (4, 4), strides=(1, 1), padding='same', activation='elu')(net_shared)
net_shared = Dropout(0.50)(net_shared)
net_shared = Conv2D(32, (4, 4), strides=(1, 1), padding='same', activation='elu')(net_shared)
net_shared = MaxPooling2D((2, 2), padding='valid')(net_shared)

net_shared = Conv2D(48, (4, 4), strides=(1, 1), padding='same', activation='elu')(net_shared)
net_shared = Dropout(0.50)(net_shared)

# COARSE CLASSIFIER
# Get the coarse predictions from fine
output_c = indep_layers(net_shared, '0')

base_model = Model(inputs=input_img, outputs=output_c)

base_model.compile(optimizer= 'adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
num_epochs = 700
base_model.fit(x_train, y_train_adj, initial_epoch=0, epochs=num_epochs, batch_size=256, validation_split=.1)
base_model.save_weights('data/models/base_model_'+str(num_epochs))

Train on 45000 samples, validate on 5000 samples
Epoch 1/700
45000/45000 [==============================] - 68s - loss: 4.7295 - acc: 0.0133 - val_loss: 4.5164 - val_acc: 0.0204
Epoch 2/700
45000/45000 [==============================] - 71s - loss: 4.4019 - acc: 0.0293 - val_loss: 4.3171 - val_acc: 0.0384
Epoch 3/700
45000/45000 [==============================] - 66s - loss: 4.2304 - acc: 0.0433 - val_loss: 4.2441 - val_acc: 0.0412
Epoch 4/700
45000/45000 [==============================] - 73s - loss: 4.1084 - acc: 0.0548 - val_loss: 4.0384 - val_acc: 0.0694
Epoch 5/700
45000/45000 [==============================] - 68s - loss: 3.9993 - acc: 0.0675 - val_loss: 4.0126 - val_acc: 0.0768
Epoch 6/700
26112/45000 [================>.............] - ETA: 29s - loss: 3.9258 - acc: 0.0770

## Fine-Tuning

**Define New Thresholding Layer**

In [14]:
from __future__ import absolute_import

from keras.layers import initializers, regularizers, constraints
from keras.engine.topology import Layer, InputSpec
from keras import backend as K
from keras.legacy import interfaces

class Threshold(Layer):
    """Thresholded Rectified Linear Unit.
    It follows:
    `f(x) = 1 for x > theta`,
    `f(x) = 0 otherwise`.
    # Input shape
        Arbitrary. Use the keyword argument `input_shape`
        (tuple of integers, does not include the samples axis)
        when using this layer as the first layer in a model.
    # Output shape
        Same shape as the input.
    # Arguments
        theta: float >= 0. Threshold value.
    """

    def __init__(self, theta=.5, **kwargs):
        super(Threshold, self).__init__(**kwargs)
        self.theta = K.cast_to_floatx(theta)

    def call(self, inputs, mask=None):
        return K.cast(K.greater(inputs, self.theta), K.floatx())

    def get_config(self):
        config = {'theta': float(self.theta)}
        base_config = super(Threshold, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

**Define Threshold and Probabalistic Averaging Functions**

In [15]:
def calc_prob_ave(fine_pred, fine_to_coarse):
    coarse_pred = Dot(1)([fine_pred,fine_to_coarse])
    coarse_pred_adj = RepeatVector(100)(coarse_pred)
    coarse_pred_adj = Flatten()(coarse_pred_adj)
    prob_ave = Multiply()([fine_to_coarse, coarse_pred_adj])

    return prob_ave

def calc_threshold(fine_pred, fine_to_coarse, shared_layer):
    coarse_pred = Dot(1)([fine_pred, fine_to_coarse])
    coarse_pred_thresholded = Threshold()(coarse_pred)
    threshold_adj = RepeatVector(4*4*48)(coarse_pred_thresholded)
    threshold_adj = Reshape((4, 4, 48))(threshold_adj)
    shared_layer_adj = Multiply()([shared_layer, threshold_adj])
    
    return shared_layer_adj

**Create an array of inputs**

In [16]:
fine_inputs = [Input(shape=([100]), dtype='float32', name='c0'), \
        Input(shape=([100]), dtype='float32', name='c1'), \
        Input(shape=([100]), dtype='float32', name='c2'), \
        Input(shape=([100]), dtype='float32', name='c3'), \
        Input(shape=([100]), dtype='float32', name='c4'), \
        Input(shape=([100]), dtype='float32', name='c5'), \
        Input(shape=([100]), dtype='float32', name='c6'), \
        Input(shape=([100]), dtype='float32', name='c7'), \
        Input(shape=([100]), dtype='float32', name='c8'), \
        Input(shape=([100]), dtype='float32', name='c9'), \
        Input(shape=([100]), dtype='float32', name='c10'), \
        Input(shape=([100]), dtype='float32', name='c11'), \
        Input(shape=([100]), dtype='float32', name='c12'), \
        Input(shape=([100]), dtype='float32', name='c13'), \
        Input(shape=([100]), dtype='float32', name='c14'), \
        Input(shape=([100]), dtype='float32', name='c15'), \
        Input(shape=([100]), dtype='float32', name='c16'), \
        Input(shape=([100]), dtype='float32', name='c17'), \
        Input(shape=([100]), dtype='float32', name='c18'), \
        Input(shape=([100]), dtype='float32', name='c19')]

**Add Fine Layers**

In [17]:
for i in range(20):
    thresholded = calc_threshold(base_model.output, fine_inputs[i], net_shared)
    fine_output = indep_layers(thresholded, str(i+1))

    prob_ave = calc_prob_ave(output_c, fine_inputs[i])

    weighted = Multiply()([fine_output, prob_ave])
    
    if i==0: output_f = weighted
    else: output_f = Add()([weighted, output_f])

**Compile new network**

In [18]:
c = fine_inputs

model = Model(inputs=[base_model.input, c[0], c[1], c[2], c[3], c[4], c[5], c[6], c[7], c[8], c[9], c[10], c[11], c[12], c[13], c[14], c[15], c[16], c[17], c[18], c[19]], 
              outputs=output_f)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer= SGD(lr=.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

**Load Coarse Weights**

In [19]:
for layer in base_model.layers:
    layer_name = layer.get_config()['name']
    if 'indep_layer' in layer_name:
        seg = layer_name.split('__')[1]
        for f_layer in model.layers:
            if seg in f_layer.get_config()['name']:
                f_layer.set_weights(layer.get_weights())

**Load Coarse-To-Fine Mappings**

In [20]:
# Get Coarse to Fine Mappings
C2K = np.loadtxt('C2K.txt', dtype=int)
C2K_dot = np.zeros((20,100))
for i in range(len(C2K)):
    C2K_dot[C2K[i], i] = 1

#np.set_printoptions(threshold=np.inf)
#print(C2K_dot)

**Fit Training Data**

In [21]:
ck = np.zeros((20, len(x_train), len(C2K_dot[0])))
for i in range(len(x_train)):
    for j in range(20):
        ck[j,i] = C2K_dot[j]

index= 0
step = 5
stop = 50

while index < stop:
    model.fit([x_train, ck[0], ck[1], ck[2], ck[3], ck[4], ck[5], ck[6], ck[7], ck[8], ck[9], ck[10], ck[11], ck[12], ck[13], ck[14], ck[15], ck[16], ck[17], ck[18], ck[19]],\
              y_train_adj, initial_epoch=index, epochs=index+step, batch_size=256, validation_split=.1)
    model.save_weights('data/models/layer_'+str(index))
    index += step

Train on 45000 samples, validate on 5000 samples
Epoch 1/5
45000/45000 [==============================] - 187s - loss: 3.2167 - acc: 0.2125 - val_loss: 3.0190 - val_acc: 0.2702
Epoch 2/5
45000/45000 [==============================] - 167s - loss: 3.1322 - acc: 0.2278 - val_loss: 3.0155 - val_acc: 0.2712
Epoch 3/5
45000/45000 [==============================] - 169s - loss: 3.1103 - acc: 0.2322 - val_loss: 2.9976 - val_acc: 0.2756
Epoch 4/5
45000/45000 [==============================] - 167s - loss: 3.0875 - acc: 0.2390 - val_loss: 2.9833 - val_acc: 0.2744
Epoch 5/5
45000/45000 [==============================] - 168s - loss: 3.0724 - acc: 0.2394 - val_loss: 2.9798 - val_acc: 0.2836
Train on 45000 samples, validate on 5000 samples
Epoch 6/10
45000/45000 [==============================] - 168s - loss: 3.0626 - acc: 0.2433 - val_loss: 2.9902 - val_acc: 0.2726
Epoch 7/10
45000/45000 [==============================] - 168s - loss: 3.0631 - acc: 0.2450 - val_loss: 2.9906 - val_acc: 0.2746
Epoc

**Evaluate on testing set**

In [ ]:
ct = np.zeros((20,len(x_test),100))
for i in range(len(x_test)):
    for j in range(20):
        ct[j,i] = C2K_dot[j]
        
model.evaluate([x_test, ck[0], ck[1], ck[2], ck[3], ck[4], ck[5], ck[6], ck[7], ck[8], ck[9], ck[10], ck[11], ck[12], ck[13], ck[14], ck[15], ck[16], ck[17], ck[18], ck[19]],\
          y_test_adj, batch_size=64, verbose=1)